In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/teaching/Desktop/Grp_22/BiometricSEAL/videoseal/

/home/teaching/Desktop/Grp_22/BiometricSEAL/videoseal


In [2]:
import os
import omegaconf
from tqdm import tqdm
import gc
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
import random
from videoseal.utils.display import save_img
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, psnr, ssim
from videoseal.augmentation import Identity, JPEG
from videoseal.modules.jnd import JND
to_tensor = torchvision.transforms.ToTensor()
to_pil = torchvision.transforms.ToPILImage()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/teaching/anaconda3/envs/videoseal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# num_imgs = -1
assets_dir = "../train/"
output_dir = "apnaDataset"
base_output_dir = "outputs"
os.makedirs(base_output_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [3]:
#write code for loadigng tensor from tensor.pt
tensor = torch.load("tensors.pt")


In [4]:
with open('tensors.txt', 'a') as f:
    for i in tensor:
        f.write(str(i) + '\n')


In [52]:
ckpt_path = "output/checkpoint025.pth"

In [53]:
model = setup_model_from_checkpoint(ckpt_path)
model.eval()
model.compile()
model.to(device)
model.blender.scaling_w *= 1

Model loaded successfully from output/checkpoint025.pth with message: <All keys matched successfully>


In [ ]:
files = [f for f in os.listdir(assets_dir) if f.endswith(".png") or f.endswith(".jpg")]
files = [os.path.join(assets_dir, f) for f in files]
# files = files[:]
for file in tqdm(files, desc=f"Processing Images"):
        # load image
        imgs = Image.open(file, "r").convert("RGB")  # keep only rgb channels
        imgs = to_tensor(imgs).unsqueeze(0).float()

        # Watermark embedding
        # timer.start()
        custom_msg = random.choice(tensor)
        #get index of the tensor
        ind = 0
        for i in range(len(tensor)):
            if torch.equal(custom_msg, tensor[i]):
                ind = i
                break
            
        outputs = model.embed(imgs, msgs = custom_msg, is_video=False, lowres_attenuation=True)
        torch.cuda.synchronize()
        # print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs
        diff = 25 * diff.abs()

        # save
        # timer.start()
        base_save_name = os.path.join(output_dir, os.path.basename(file).replace(".png", ""))
        # print(base_save_name)
        # save_img(imgs[0], f"{base_save_name}_ori.png")
        save_img(imgs_w[0], f"{base_save_name}_wm.png")
        # save_img(diff[0], f"{base_save_name}_diff.png")

        with open("Dataset.txt", 'a') as f:
                f.write(f"{base_save_name}_wm.png;{ind};\n")

        # Metrics
        imgs_aug = imgs_w
        outputs = model.detect(imgs, is_video=False)
        metrics = {
            "file": file,
            "bit_accuracy": bit_accuracy(
                outputs["preds"][:, 1:],
                msgs
            ).nanmean().item(),
            "psnr": psnr(imgs_w, imgs).item(),
            "ssim": ssim(imgs_w, imgs).item()
        }

In [4]:
tensor = torch.load("./tensors.pt")

In [5]:
print(tensor[0].shape)

torch.Size([128])


In [54]:
#load image in tensor
wm_img = Image.open("apnaDataset/sa_11188.jpg_wm.png", "r").convert("RGB")  # keep only rgb channels
wm_img = to_tensor(wm_img).unsqueeze(0).float().to(device)

In [55]:
print(wm_img.shape)

torch.Size([1, 3, 512, 770])


In [56]:
ori_im = Image.open("../train/sa_11188.jpg", "r").convert("RGB")
ori_im = to_tensor(ori_im).unsqueeze(0).float().to(device)
ori_im.shape

torch.Size([1, 3, 512, 770])

In [7]:
preds = torch.tensor([0.5457, 0.3809, 0.3267, 0.6044, 0.2729, 0.6024, 0.4957, 0.2257, 0.1810,
         0.4398, 0.2844, 0.4610, 0.5749, 0.2714, 0.4818, 0.6489, 0.4618, 0.4516,
         0.6742, 0.2588, 0.5161, 0.4734, 0.2816, 0.3328, 0.5475, 0.1698, 0.6490,
         0.4243, 0.8023, 0.4102, 0.6116, 0.3908, 0.5433, 0.5134, 0.7281, 0.6355,
         0.4217, 0.6171, 0.7323, 0.5519, 0.8825, 0.5006, 0.7277, 0.2345, 0.5104,
         0.3111, 0.3415, 0.5947, 0.4191, 0.5248, 0.5995, 0.5193, 0.5391, 0.3411,
         0.8264, 0.6773, 0.6044, 0.6403, 0.7414, 0.3697, 0.6506, 0.4235, 0.3757,
         0.6038, 0.4386, 0.7143, 0.6640, 0.4485, 0.3084, 0.1317, 0.4364, 0.4075,
         0.5763, 0.4766, 0.4714, 0.0781, 0.6287, 0.3901, 0.4174, 0.5017, 0.3572,
         0.6354, 0.4958, 0.8214, 0.5654, 0.6796, 0.4098, 0.4678, 0.3324, 0.6267,
         0.2417, 0.5854, 0.2926, 0.6071, 0.6794, 0.2375, 0.3143, 0.5138, 0.3482,
         0.3525, 0.8601, 0.1500, 0.3864, 0.4100, 0.4920, 0.5449, 0.6572, 0.3407,
         0.3961, 0.2868, 0.5889, 0.4995, 0.3970, 0.6622, 0.5342, 0.6001, 0.7509,
         0.3797, 0.4014, 0.4548, 0.1398, 0.1208, 0.5474, 0.2866, 0.7947, 0.7861,
         0.3184, 0.2924, 0.5212])[1:]

In [8]:
preds.shape

torch.Size([128])

In [9]:
def cos(a, b):
    import torch.nn.functional as F
    a = a.view(1, -1)
    b = b.view(1, -1)
    cos = F.cosine_similarity(a, b)
    return cos.item()


In [12]:
tensors = torch.load("tensors.pt")
for i in tensors:
    print(cos(i, preds))

0.8234018683433533
0.7873243093490601
0.8287986516952515
0.8181370496749878
0.781493067741394
0.8249193429946899
0.8327418565750122
0.8461710214614868
0.8308339715003967
0.8441811203956604
0.8323599100112915
0.7941389083862305
0.8211923837661743
0.8554711937904358
0.8202939033508301
0.7995548844337463
0.8642961978912354
0.7939215302467346
0.8037793040275574
0.7946099638938904
0.8510743975639343
0.8285080194473267
0.7949255704879761
0.7996906638145447
0.8130337595939636
0.8097113370895386
0.7858120799064636
0.8095712661743164
0.8045985698699951
0.8316817283630371
0.8374031186103821
0.8243930339813232
0.8464142680168152
0.8295799493789673
0.8304649591445923
0.8458789587020874
0.8277631998062134
0.8117071986198425
0.8289353847503662
0.7973959445953369
0.8088787794113159
0.8365012407302856
0.8212382197380066
0.832080066204071
0.796597957611084
0.8325291275978088
0.7974777817726135
0.8203443884849548
0.8073456287384033
0.8093833923339844
